#This version can ONLY be used for Alvis stock, the input data source is IB statement, the output is pnl csv report, logic is first in first out 

In [2]:
import pandas as pd
import csv
from datetime import datetime, timedelta
from collections import deque
from blpapi import Session, SessionOptions
from blp import blp

In [3]:
# read the data from ib statement file
ib_data = []
with open('U12393385_20250101_20250206.csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    for row in reader:
        ib_data.append(row)

In [ ]:
def convert_ib_code_to_bb_ticker(row):
    """manually maintain a mapping of IB codes to Bloomberg tickers"""
    if row['Asset Category'] == 'Stocks' and row['Currency'] == 'USD':
        return row['Symbol'] + ' US Equity'
    # need change
    # if row['Asset Category'] == 'Futures':
    #     return row['Symbol'] + ' Comdty'
    pass

In [ ]:
# data cleaning
trades = []
for row in ib_data:
    if row[0] == 'Trades':
        trades.append(row)
# crete dataframe from trades
df = pd.DataFrame(trades[1:], columns=trades[0])
df['BB_Symbol'] = df.apply(convert_ib_code_to_bb_ticker, axis=1)
df.drop(columns=['Trades', 'Header', 'DataDiscriminator','Realized P/L','MTM P/L','Code'], inplace=True)
stock_df = df[df['Asset Category'] == 'Stocks']
futures_df = df[df['Asset Category'] == 'Futures']
#drop the rows that have no value in the Date/Time column
stock_df = stock_df[stock_df['Date/Time'] != '']
stock_df['Date/Time'] = pd.to_datetime(stock_df['Date/Time'])
futures_df = futures_df[futures_df['Date/Time'] != '']
futures_df['Date/Time'] = pd.to_datetime(futures_df['Date/Time'])
columns_to_convert = ['Quantity', 'T. Price', 'C. Price','Comm/Fee']
for col in columns_to_convert:
    stock_df[col] = pd.to_numeric(stock_df[col].str.replace(',', ''))


In [6]:
stock_df

,Asset Category,Currency,Symbol,Date/Time,Quantity,T. Price,C. Price,Proceeds,Comm/Fee,Basis,BB_Symbol
0,Stocks,USD,SGOV,2025-01-08 05:37:10,15184,100.41,100.41,-1524625.44,-102.303301,1524727.743300841,SGOV US Equity
1,Stocks,USD,SGOV,2025-01-31 11:41:08,-8996,100.69,100.68,905807.24,-45.751379,-902569.3436,SGOV US Equity
2,Stocks,USD,SGOV,2025-01-31 14:52:27,-23576,100.69,100.68,2373867.44,-108.514540,-2365390.774784,SGOV US Equity
3,Stocks,USD,SGOV,2025-01-31 19:46:02,-37742,100.68,100.68,3799864.56,-366.190868,-3786594.285461001,SGOV US Equity
4,Stocks,USD,SGOV,2025-02-04 10:55:28,46715,100.36,100.36,-4688317.4,-101.697699,4688419.097699339,SGOV US Equity
5,Stocks,USD,SGOV,2025-02-04 10:55:29,23815,100.36,100.36,-2390073.4,-82.007289,2390155.407289088,SGOV US Equity


In [4]:
# blb test
bquery = blp.BlpQuery().start() 
df = bquery.bdh(["SGOV US Equity"], ["PX_LAST"], "20250214", "20250217")[['date', 'PX_LAST']]
df

,date,PX_LAST
0,2025-02-14,100.5


In [88]:
def get_close_price_from_bloomberg(ticker, query_date):
    """
    eg:'SGOV US Equity','2025-02-11'
    when query_date is not specified, treat it as the latest close price
    """
    if query_date is None:
        query_date = datetime.now()
    else:
        query_date = datetime.strptime(query_date, '%Y-%m-%d')
    if 'US Equity' in ticker:
        if query_date.weekday() == 0:
            query_date = query_date - timedelta(days=3)
        else:
            query_date = query_date - timedelta(days=1) 
    else: # Asia market
        if query_date.time() < datetime.strptime('17:00:00', '%H:%M:%S').time():
            query_date = query_date - timedelta(days=1)
    query_date = query_date.strftime('%Y%m%d')
            
    bquery = blp.BlpQuery().start()
    market_price = float('nan')  
    try:
        df = bquery.bdh([ticker], ['PX_LAST'], query_date, query_date)[['date', 'PX_LAST']]
        if df.empty:
            print(f"{ticker} at {query_date} not found!!")
        else:
            market_price = df['PX_LAST'].values[0]
    except Exception as e:
        print(f"{ticker} at {query_date} not found!!")
        
    finally:
        bquery.stop()
        
    return market_price

In [112]:
class TradingPosition:
    '''define a class to record each trade and calculate pnl'''
    def __init__(self):
        self.position = deque()  
        self.realized_pnl = 0

    def make_trade_record(self, price, quantity, fee):
        # fee is not divided，it represents the cost at the trade happens
        self.position.append((price, quantity, fee))  
        self.realized_pnl += fee    
        
    def get_realized_pnl(self, price, quantity, fee):
        # get realized pnl when position closed 
        # handle both long and short
        # this function is used for offsetting the position
        unclosed_quantity = quantity
        
        if self.position:
            if self.position[0][1]>0: # if position is long
                while (unclosed_quantity < 0 and self.position): #  when selling and still have positive position
                    buy_price, buy_quantity, buy_fee = self.position[0] # get previous buy price
                    if buy_quantity > (- unclosed_quantity): # when have enough position
                        self.realized_pnl += (price - buy_price) * (- unclosed_quantity) 
                        print('realized_pnl1',self.realized_pnl)
                        # offset to 1 after selling
                        self.position.popleft()
                        self.position.pop()
                        self.position.appendleft((buy_price, buy_quantity + unclosed_quantity, buy_fee))
                        unclosed_quantity = 0
                        print("HERE1",self.position)
                    else: # when don't have enough position
                        self.realized_pnl += (price - buy_price) * buy_quantity 
                        print('realized_pnl2',self.realized_pnl)
                        unclosed_quantity += buy_quantity
                        # offset 
                        self.position.popleft() 
                        self.position.pop()
                        print("HERE2",self.position)
                        if(unclosed_quantity < 0 and self.position):
                            self.position.append((price, unclosed_quantity, fee))  
                # add new position after offsetting
                if unclosed_quantity < 0 and not self.position:
                    self.position.append((price, unclosed_quantity, fee))  
            else: # if position is short
                while (unclosed_quantity > 0 and self.position): #when buying and still have position
                    sell_price, sell_quantity, sell_fee = self.position[0] # get previous selling price
                    if (- sell_quantity) > unclosed_quantity: # when have enough position
                        self.realized_pnl += (sell_price - price) * unclosed_quantity 
                        print('realized_pnl3',self.realized_pnl)
                        self.position.popleft()
                        self.position.pop()
                        self.position.appendleft((sell_price, sell_quantity + unclosed_quantity, sell_fee))
                        unclosed_quantity = 0
                        print("HERE3",self.position)
                    else:  # when don't have enough position
                        self.realized_pnl += (sell_price - price) * (- sell_quantity) 
                        print('realized_pnl4',self.realized_pnl)
                        unclosed_quantity += sell_quantity
                        # offset
                        self.position.popleft() 
                        self.position.pop()
                        print("HERE4",self.position)
                        if(unclosed_quantity > 0 and self.position):
                            self.position.append((price, unclosed_quantity, fee))
                if unclosed_quantity > 0: # add new position after offsetting
                    self.position.append((price, unclosed_quantity, fee))
                                                    
        print("HERE5",self.position)
        print("realized_pnl",self.realized_pnl)     
        print("----------------------------------------")
        
        return self.realized_pnl
    
    def get_unrealized_pnl(self, current_price):
        unrealized_pnl = 0
        if self.position:
            for price, quantity, fee in self.position:
                unrealized_pnl += (current_price - price) * quantity
        else:
            pass
        return float(unrealized_pnl)
    
    def current_position_df(self):
        df = pd.DataFrame(list(self.position), columns=['price, quantity, fee'])
        return df

In [ ]:
def calculation(df, start_date, end_date):
    '''calculate pnl, exposure and return in a period, if date is not specified, treat it as the whole period.'''
    trading_positions = {}  
    realized_pnl = 0
    unrealized_pnl = 0
    
    df = df[(df['Date/Time'] >= start_date) & (df['Date/Time'] <= end_date)]
    df_sorted = df.sort_values(by=['BB_Symbol', 'Date/Time'])
    for bb_symbol, group in df_sorted.groupby('BB_Symbol'):
        if bb_symbol not in trading_positions:
            trading_positions[bb_symbol] = TradingPosition()
    
        for idx, row in group.iterrows():
            quantity = row['Quantity']
            price = row['T. Price']
            fee = row['Comm/Fee']
            trading_positions[bb_symbol].make_trade_record(price, quantity, fee)
            realized_pnl = trading_positions[bb_symbol].get_realized_pnl(price, quantity, fee)
        
    # calculate unrealized pnl based on current position  
    for bb_symbol in trading_positions.keys():
        current_price = get_close_price_from_bloomberg(bb_symbol, end_date)
        unrealized_pnl = trading_positions[bb_symbol].get_unrealized_pnl(current_price)
    
    return realized_pnl, unrealized_pnl



In [ ]:
realized_pnl, unrealized_pnl = calculation(stock_df, '2025-01-01','2025-02-14')

HERE5 deque([(100.41, 15184, -102.303300841)])
realized_pnl -102.303300841
----------------------------------------
realized_pnl1 2370.82531967501
HERE1 deque([(100.41, 6188, -102.303300841)])
HERE5 deque([(100.41, 6188, -102.303300841)])
realized_pnl 2370.82531967501
----------------------------------------
realized_pnl2 3994.9507795830173
HERE2 deque([])
HERE5 deque([(100.69, -17388, -108.514540092)])
realized_pnl 3994.9507795830173
----------------------------------------
HERE5 deque([(100.69, -17388, -108.514540092), (100.68, -37742, -366.190868061)])
realized_pnl 3628.7599115220173
----------------------------------------
realized_pnl4 9265.102212183987
HERE4 deque([(100.68, -37742, -366.190868061)])
realized_pnl3 18649.742212184203
HERE3 deque([(100.68, -8415, -366.190868061)])
HERE5 deque([(100.68, -8415, -366.190868061)])
realized_pnl 18649.742212184203
----------------------------------------
realized_pnl4 21260.534923096264
HERE4 deque([])
HERE5 deque([(100.36, 15400, -82.007

In [110]:
realized_pnl, unrealized_pnl

(21260.534923096264, 2156.0000000000086)

In [ ]:
# WTD
last_wtd_end_pnl = calculation(stock_df, '2025-01-01', '2025-02-14')
wtd_pnl = realized_pnl-last_wtd_end_pnl[0], unrealized_pnl-last_wtd_end_pnl[1]
# MTD
last_mtd_end_pnl = calculation(stock_df, '2025-01-01', '2025-01-31')
mtd_pnl = realized_pnl-last_mtd_end_pnl[0], unrealized_pnl-last_mtd_end_pnl[1]
# YTD
last_ytd_end_pnl = calculation(stock_df, '2024-01-01', '2024-12-31')
ytd_pnl = realized_pnl-last_ytd_end_pnl[0], unrealized_pnl-last_ytd_end_pnl[1]

In [ ]:
# output in csv file
with open('pnl.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['PnL', 'WTD', 'MTD', 'YTD'])
    writer.writerow(['Realized', realized_pnl, wtd_pnl[0], mtd_pnl[0], ytd_pnl[0]])
    writer.writerow(['Unrealized', unrealized_pnl, wtd_pnl[1], mtd_pnl[1], ytd_pnl[1]])